In [1]:
import akantu
# import numpy as np
print("Akantu imported successfully!")
print("Available attributes:")
print([attr for attr in dir(akantu) if not attr.startswith('_')][:10])

Akantu imported successfully!
Available attributes:
[]


In [2]:
try:
    # Check if there is an initialization method
    if hasattr(akantu, 'initialize'):
        akantu.initialize()
        print("✓ Akantu initialized successfully")
    else:
        print("!! Akantu does not require explicit initialization")
    
    # Try to create a mesh
    if hasattr(akantu, 'Mesh'):
        mesh = akantu.Mesh(2)
        print(f"✓ Successfully created 2D mesh")
        print(f"  - Spatial dimension: {mesh.getSpatialDimension()}")
    else:
        print("❌ Mesh class not available")
        
except Exception as e:
    print(f"Error: {e}")

!! Akantu does not require explicit initialization
❌ Mesh class not available


In [3]:
print("=== Akantu Feature Exploration ===")

# List main classes
main_classes = [attr for attr in dir(akantu) 
                if not attr.startswith('_') and attr[0].isupper()]

print(f"Main classes ({len(main_classes)}):")
for cls in sorted(main_classes)[:15]:  # Show first 15
    print(f"  - {cls}")

if len(main_classes) > 15:
    print(f"  ... {len(main_classes) - 15} more classes")

=== Akantu Feature Exploration ===
Main classes (0):


In [4]:
def first_akantu_example():
    """First Akantu Example: Create mesh and basic operations"""
    
    try:
        print("=== First Akantu Example ===\n")
        
        # Initialize (if needed)
        if hasattr(akantu, 'initialize'):
            akantu.initialize()
            print("1. ✓ Akantu initialized")
        
        # Create mesh
        mesh = akantu.Mesh(2)  # 2D mesh
        print("2. ✓ Created 2D mesh")
        
        # Check mesh node attribute
        if hasattr(mesh, 'getNodes'):
            nodes = mesh.getNodes()
            print("3. ✓ Retrieved node array")
            print(f"   - Node array type: {type(nodes)}")
            
        # Check other mesh attributes
        mesh_attrs = [attr for attr in dir(mesh) if not attr.startswith('_')]
        print(f"4. Mesh available methods: {mesh_attrs[:10]}...")  # Show first 10
        
        # Try to get spatial dimension
        if hasattr(mesh, 'spatial_dimension'):
            print(f"5. Spatial dimension: {mesh.spatial_dimension}")
        elif hasattr(mesh, 'getDimension'):
            print(f"5. Dimension: {mesh.getDimension()}")
        else:
            print("5. Cannot directly get spatial dimension")
        
        print(f"\nAkantu example ran successfully!")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

# Run example
first_akantu_example()

=== First Akantu Example ===

❌ Error: module 'akantu' has no attribute 'Mesh'


Traceback (most recent call last):
  File "/var/folders/3_/nmxnf4m14bq6l5lt8nd299xh0000gp/T/ipykernel_49100/2577840985.py", line 13, in first_akantu_example
    mesh = akantu.Mesh(2)  # 2D mesh
           ^^^^^^^^^^^
AttributeError: module 'akantu' has no attribute 'Mesh'


In [5]:
def create_gmsh_file(rupture_position=55.0):
    """
    Create a correct GMSH mesh file for Akantu
    """
    
    # Parameters
    W, H, DEPTH = 100.0, 160.0, 50.0
    mesh_size = 10.0
    
    filename = f"block_mesh_{int(rupture_position)}.msh"
    
    # Corrected GMSH format
    gmsh_content = f"""$MeshFormat
2.2 0 8
$EndMeshFormat
$PhysicalNames
7
2 1 "bottom"
2 2 "top" 
2 3 "left"
2 4 "right"
2 5 "front"
2 6 "back"
3 7 "block"
$EndPhysicalNames
$Nodes
8
1 -{W/2} -{H/2} 0
2 {W/2} -{H/2} 0
3 {W/2} {H/2} 0
4 -{W/2} {H/2} 0
5 -{W/2} -{H/2} {DEPTH}
6 {W/2} -{H/2} {DEPTH}
7 {W/2} {H/2} {DEPTH}
8 -{W/2} {H/2} {DEPTH}
$EndNodes
$Elements
1
1 5 2 7 7 1 2 3 4 5 6 7 8
$EndElements
"""
    
    with open(filename, 'w') as f:
        f.write(gmsh_content)
    
    print(f"Created GMSH file: {filename}")
    return filename

In [6]:
# 先檢查 Mesh 的實際方法
import akantu.py11_akantu as aka

mesh = aka.Mesh(3)
print("All Mesh methods:")
methods = [method for method in dir(mesh) if not method.startswith('_')]
for i, method in enumerate(methods):
    print(f"{i:2d}. {method}")

# 特別檢查元素相關的方法
element_methods = [method for method in methods if 'element' in method.lower() or 'Element' in method]
print(f"\nElement-related methods: {element_methods}")

# 檢查節點相關的方法
node_methods = [method for method in methods if 'node' in method.lower() or 'Node' in method]
print(f"\nNode-related methods: {node_methods}")

ImportError: dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/Akantu/lib/python3.12/site-packages/akantu/py11_akantu.cpython-312-darwin.so, 0x0002): symbol not found in flat namespace '_SCOTCH_errorPrint'

In [ ]:
#!/usr/bin/env python3
"""
Alternative approach - maybe we need to load materials from input file
"""

import akantu.py11_akantu as aka
import numpy as np
import os

def create_material_input_file():
    """
    Create a material input file (.dat) for Akantu
    """
    
    filename = "material.dat"
    
    # Akantu material file format
    material_content = """
material elastic [
    name = granite
    rho = 2.65426e-9  # kg/mm^3
    E = 30000         # MPa  
    nu = 0.25         # Poisson ratio
]
"""
    
    with open(filename, 'w') as f:
        f.write(material_content)
    
    print(f"Created material file: {filename}")
    return filename

def try_input_file_approach():
    """
    Try loading materials from input file
    """
    
    print("TRYING INPUT FILE APPROACH")
    print("="*50)
    
    # Create material file
    material_file = create_material_input_file()
    
    try:
        # Check if we can initialize with input file
        print("Trying to initialize with material file...")
        
        # Method 1: Try initialize function (like C++ examples)
        try:
            aka.initialize(material_file)
            print("✓ Global initialization successful")
        except Exception as e:
            print(f"Global initialization failed: {e}")
        
        # Create mesh and model
        mesh_file = create_simple_block_mesh(55)
        mesh = aka.Mesh(3)
        mesh.read(mesh_file)
        
        model = aka.SolidMechanicsModel(mesh)
        
        # Try initialization
        model.initFull()
        
        nb_materials = model.getNbMaterials()
        print(f"Number of materials after init: {nb_materials}")
        
        if nb_materials > 0:
            material = model.getMaterial(0)
            print("✓ Got material successfully!")
            return True
            
    except Exception as e:
        print(f"Input file approach failed: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Cleanup
        for f in [material_file, "simple_block_55.msh"]:
            if os.path.exists(f):
                os.remove(f)
    
    return False

def create_simple_block_mesh(rupture_position=55.0):
    """Simple mesh creation"""
    W, H, DEPTH = 50.0, 80.0, 25.0
    filename = f"simple_block_{int(rupture_position)}.msh"
    
    gmsh_content = f"""$MeshFormat
2.2 0 8
$EndMeshFormat
$PhysicalNames
1
3 1 "block"
$EndPhysicalNames
$Nodes
8
1 -{W/2} -{H/2} 0
2 {W/2} -{H/2} 0  
3 {W/2} {H/2} 0
4 -{W/2} {H/2} 0
5 -{W/2} -{H/2} {DEPTH}
6 {W/2} -{H/2} {DEPTH}
7 {W/2} {H/2} {DEPTH}
8 -{W/2} {H/2} {DEPTH}
$EndNodes
$Elements
1
1 5 2 1 1 1 2 3 4 5 6 7 8
$EndElements
"""
    
    with open(filename, 'w') as f:
        f.write(gmsh_content)
    
    return filename

def check_alternative_model_types():
    """
    Check if there are other model types available
    """
    
    print("CHECKING ALTERNATIVE MODEL TYPES")
    print("="*50)
    
    # Check what model types are available
    model_classes = [attr for attr in dir(aka) if 'Model' in attr and not attr.startswith('_')]
    print(f"Available model classes: {model_classes}")
    
    # Check other physics models
    physics_classes = [attr for attr in dir(aka) if any(x in attr for x in ['Heat', 'Structural', 'Contact', 'Phase'])]
    print(f"Other physics classes: {physics_classes}")
    
    # Try a simpler approach - maybe we don't need materials for a basic test?
    try:
        print("\nTrying minimal model without explicit materials...")
        mesh = aka.Mesh(3)
        
        # Create minimal mesh in memory (if possible)
        model = aka.SolidMechanicsModel(mesh)
        
        # Check what happens if we init without materials
        model.initFull()
        
        print("✓ Model initialized without explicit materials")
        return True
        
    except Exception as e:
        print(f"Minimal model failed: {e}")
        return False

def main():
    """
    Try multiple approaches
    """
    
    print("AKANTU ALTERNATIVE APPROACHES")
    print("="*70)
    
    # First check registerNewMaterial signature
    print("1. CHECKING REGISTERNEWMATERIAL API...")
    # (paste the first code block here)
    
    # Then try input file approach  
    print("\n2. TRYING INPUT FILE APPROACH...")
    input_file_success = try_input_file_approach()
    
    # Check alternative model types
    print("\n3. CHECKING ALTERNATIVE APPROACHES...")
    minimal_success = check_alternative_model_types()
    
    if input_file_success:
        print("\n✓ Input file approach worked!")
    elif minimal_success:
        print("\n✓ Minimal approach worked!")
    else:
        print("\n❌ All approaches failed - may need different Akantu version or approach")

if __name__ == "__main__":
    main()